In [5]:
import numpy as np
import pandas as pd
import csv
import random

# Инициализация

In [15]:
drop_column = ['№п/п', 'Дата рождения', 'Учебное заведение', 'Населенный пункт по прописке', 'Основание поступления']
gender_dict = {'Мужской': 0.95,'Женский': 0.05}

# подготовленный датасет
prepare_data = []
# количество нейронов кохонена
kohonen_neuron_count = 30
# скорость обучения
theta = 0.7

# Считывание данных из файла и их подготовка

In [16]:
df = pd.read_csv('data/input.csv', sep=',')
df.drop(drop_column, axis = 1, inplace = True)

# переводим пол в числовой формат, т. к. это будет один из признаков
df['Пол'].replace(gender_dict, inplace = True) 

# приводим баллы по ЕГЭ к диапозону от 0 до 1
df.astype({'Баллы ЕГЭ': 'float'}).dtypes
df['Баллы ЕГЭ'] /= 100

fios = df['ФИО'].unique()
subjects = df['Предметы ЕГЭ'].unique()
profs = df['Специальность/направление'].unique()
np.where(profs == 'Экономика')[0].item(0)
for el in fios:
    student = df[df['ФИО'] == el]
    features = np.zeros((6))
    for i in range(len(subjects)):
        res = student.loc[student['Предметы ЕГЭ'] == subjects[i], 'Баллы ЕГЭ'].values
        features[i] = res[0] if len(res) else 0
    features[5] = student.iloc[0]['Пол']
    student_data = dict(fio = el, features = features, 
                    result = np.where(profs == student.iloc[0]['Специальность/направление'])[0].item(0))
    prepare_data.append(student_data)

# Обучение

## Кохонен

In [21]:
def getKohonenWinnerIndex(fio, x, neuron_counts, w):
    result = 0
    max_net = -1
    for i in range(neuron_counts):
        net = 0
        for j in range(len(x)):
            net += (x[j] - w[j][i]) ** 2
        if (net > max_net):
            result = i
            max_net = net
    return result

In [22]:
#количество нейронов выходного слоя (слоя Гроссберга) = количество проффессий 
grossberg_neuron_counts = len(profs)

#инициализация начальных весов
weights = np.zeros((len(subjects) + 1, kohonen_neuron_count))
for i in range(len(subjects) + 1):
    for j in range(kohonen_neuron_count):
        weights[i, j] = random.uniform(0.01, 0.99)


for student in prepare_data:
    winner_index = getKohonenWinnerIndex(student['fio'], student['features'], kohonen_neuron_count, weights)

print(weights[0][winner_index])
for i in range(len(subjects) + 1):
    weights[i][winner_index] = weights[i][winner_index] - theta * (weights[i][winner_index] - student['features'][i])
print(weights[0][winner_index])

0.025931911123286827
0.511779573336986


In [ ]:
2 ** 3